# APS1070
#### Tutorial 2 - Anomaly Detection Algorithm using Gaussian Mixture Model 


In this part of tutorial, we will implement an anomaly detection algorithm using the Gaussian model to detect anomalous behavior in a 2D dataset first and then a high-dimensional dataset.

Loading relevant libraries and the dataset

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

X, y_true = make_blobs(n_samples=400, centers=1,
                       cluster_std=0.60, random_state=0)
X_append, y_true_append = make_blobs(n_samples=10,centers=1,
                                    cluster_std=5,random_state=0)
X = np.vstack([X,X_append])
y_true = np.hstack([y_true, [1 for _ in y_true_append]])
X = X[:, ::-1] # flip axes for better plotting
plt.scatter(X[:,0],X[:,1],marker="x");

Here we've manufactured a dataset where some points are visibly outliers from the main distribution.

We can see this from looking at the plot, but how do we robustly identify the outliers? 

That's where a Gaussian estimation comes in. For this dataset, we only need a single Gaussian, for which we are gonna calculate the mean and standard deviation. Then, we're able to find the points that don't seem likely to have originated from that distribution - these are our outliers!

First, we need to calculate the mean and variance for our data. Complete the function below to generate these values using these formulas:

$$\mu = \frac{1}{m} \sum_{i=1}^{m}X_i$$

$$\sigma^2 = \frac{1}{m} \sum_{i=1}^{m}(X_i-\mu)^2$$

In [ ]:
def estimateGaussian(X):
    """
     This function estimates the parameters of a Gaussian distribution using the data in X
    """
    
    m = X.shape[0]
    
    #compute mean of X
    sum_ = np.sum(X,axis=0)
    mu = 1/m *sum_
    
    # compute variance of X
    var = 1/m * np.sum((X - mu)**2,axis=0)
    
    return mu,var
mu, sigma2 = estimateGaussian(X)

Now, we will calculate for each point in X, the probability of the distribution $N(\mu,\sigma^2)$ generating that point randomly. This has been completed for you, although it is important to understand how the calculation of the PDF works.

In [ ]:
def multivariateGaussian(X, mu, sigma2):
    """
    Computes the probability density function of the multivariate gaussian distribution.
    """
    k = len(mu)
    
    sigma2=np.diag(sigma2)
    X = X - mu.T
    p = 1/((2*np.pi)**(k/2)*(np.linalg.det(sigma2)**0.5))* np.exp(-0.5* np.sum(X @ np.linalg.pinv(sigma2) * X,axis=1))
    return p
p = multivariateGaussian(X, mu, sigma2)

Now that we have the probability of each point in the dataset, we can plot these on the original scatterplot:

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(X[:,0],X[:,1],marker="x",c=p,cmap='viridis');
plt.colorbar();

We're getting closer to the point where we can programmatically  identify our outliers for a single Gaussian distribution. The last step is to identify a value for $p$, below which we consider a point to be an outlier. We term this $\epsilon$.

In [ ]:
#Choose a value for Threshold
print (p.min() , p.max())


In [ ]:
_ = plt.hist (p, bins = 500)

In [ ]:
tr = 0.01

Now we'll highlight on the scatter plot all points that are below $\epsilon$:

In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(X[:,0],X[:,1],marker="x",c=p,cmap='viridis');
# Circling of anomalies
outliers = np.nonzero(p<tr)[0]
plt.scatter(X[outliers,0],X[outliers,1],marker="o",facecolor="none",edgecolor="r",s=70);

Try different values for threshold and report Precision, Recall and F1 Score. 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
tr_list = [0.001, 0.01, 0.02, 0.03, 0.1 , 0.2 , 1]
for tr in tr_list:
    pr = precision_score(y_true, p<=tr)
    rc = recall_score(y_true, p<=tr)
    f1 = f1_score(y_true, p<=tr)
    print ("Threshold:" , "{:.3f}".format(tr), " Precision:" , "{:.2f}".format(pr), " Recall: " , "{:.2f}".format(rc) , " F1 Score: ", "{:.2f}".format(f1))
    


You may have noticed that in this example, we are training and testing on the _entire_ dataset. This is absolutely not standard practice! You should _always_ split into a training and testing set. However, the reason that we can get away with this here is that we don't actually use labels at all during training - this is an _unsupervised_ machine learning task. Unsupervised learning methods are beneficial for anomaly detection because in the real world (i.e. testing!) we might come across types of outliers that we didn't see during training. We want to use a method that can handle this, and unsupervised methods are often better suited to this type of domain.

For the next section, we'll move to a Mixture of Gaussian models. Take a look at the following dataset:

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
X, y_true = make_blobs(n_samples=400, centers=5,
                       cluster_std=0.60, random_state=1)
X_append, y_true_append = make_blobs(n_samples=50,centers=5,
                                    cluster_std=5,random_state=1)
X = np.vstack([X,X_append])
y_true = np.hstack([[0 for _ in y_true], [1 for _ in y_true_append]])

X_train, X_test, y_train, y_test = train_test_split(X, y_true, test_size=0.33, random_state=1, shuffle=True)

plt.figure(figsize=(8,6))
plt.scatter(X_train[:,0],X_train[:,1],marker="x");

Okay, we have more than one cluster centre now. So what? Let's just ignore that and use the same model as before. 


In [ ]:
mu, sigma2 = estimateGaussian(X_train)
p = multivariateGaussian(X_train, mu, sigma2)
plt.figure(figsize=(8,6))
plt.scatter(X_train[:,0],X_train[:,1],marker="x",c=p,cmap='viridis');
outliers = np.nonzero(p<0.001)[0]
plt.scatter(X_train[outliers,0],X_train[outliers,1],marker="o",facecolor="none",edgecolor="r",s=70);

In [ ]:
mu, sigma2 = estimateGaussian(X_train)
p = multivariateGaussian(X_test, mu, sigma2)
plt.figure(figsize=(8,6))
plt.scatter(X_test[:,0],X_test[:,1],marker="x",c=p,cmap='viridis');
outliers = np.nonzero(p<0.001)[0]
plt.scatter(X_test[outliers,0],X_test[outliers,1],marker="o",facecolor="none",edgecolor="r",s=70);


Uh oh. This model performs terribly. It's fit the mean to a section of space where we don't have _any_ points, and it has absolutely no idea which points are outliers! This was probably pretty obvious to you though. We need to move to a Mixture of Gaussians model - one in which we use multiple Gaussians to fit the data. We'll use `sklearn.mixture.GaussianMixture` to do this - or rather you will! Use the documentation, found [here](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture) to initialise and fit a `GaussianMixture` object called `gm` in the cell below.

In [ ]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components = 5,
                    covariance_type = 'full', random_state=0, )
gm.fit(X_train)

Now we can use the method `gm.score_samples()` which gives a score based on how likely a point is to have been generated by any cluster:

In [ ]:
plt.figure(figsize=(10,10))
c = gm.score_samples(X_train)
plt.scatter(X_train[:,0],X_train[:,1],c=gm.score_samples(X_train),cmap='viridis',marker='x')

threshold= -4.8

outliers = np.nonzero(c<threshold)[0]
plt.scatter(X_train[outliers,0],X_train[outliers,1],marker="o",facecolor= "none",edgecolor="r",s=70)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
c = gm.score_samples(X_test)
plt.scatter(X_test[:,0],X_test[:,1],c=gm.score_samples(X_test),cmap='viridis',marker='x')

threshold= -4.8

outliers = np.nonzero(c<threshold)[0]
plt.scatter(X_test[outliers,0],X_test[outliers,1],marker="o",facecolor= "none",edgecolor="r",s=70)
plt.show()

What is the variable "`threshold`"? ______________ Why is it negative? __________________________
 
Now we can use the method `gm.predict_proba()` to spot the points in each of the clusters.

In [ ]:
Non_Outliers=np.nonzero(c>=threshold)[0]
X_t=X_train[Non_Outliers]

In [ ]:
plt.figure(figsize=(10,10))
for i in range(5):
    plt.subplot(3,2,i+1)
    plt.scatter(X_t[:,0],X_t[:,1],c=gm.predict_proba(X_t)[:,i],cmap='viridis',marker='x')

1] What do functions `gm.score_samples()` and `gm.predict_proba()` return? ___________ 

2] Why it was important to run them in above sequence? ____________________

3] What is the difference between the two function?_______________________________


Our Mixture of Gaussians model is powerful! Not only is it unsupervised, it can both classify points into one of the K clusters we have, _and_ it can help us with our ultimate goal of identifying outlier points! We can do this by finding the points that no cluster wants to claim for itself.

In the cells below, be are going to try two methods to determine the threshold and visit ROC and AUC. 

In [ ]:
p_gm = gm.score_samples(X_train)
_ = plt.hist (p_gm , bins = 100)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

p_gm = gm.score_samples(X_train) #score_samples will compute the weighted log probabilities for each sample

print ("Method 1: ")

for tr in [  -20, -10 ,-7, -6 ,  -5 ]:
   precision = precision_score(y_train, p_gm < tr) #Here, we compare y_train labels to our picks using precision
   recall = recall_score(y_train, p_gm < tr) #Here, we compare y_train labels to our picks using recall
   print('For threshold of ',tr,' \t precision: ', '%.3f' % precision,' recall: ', '%.3f' % recall) 




print ("\n\nMethod 2: ")

for i in [1, 10, 20, 30, 100 , 120, 130]: 
   tr = sorted(p_gm)[i] #We sort the points by probability
   precision = precision_score(y_train, p_gm < tr) 
   recall = recall_score(y_train, p_gm < tr) 
   print('For k: ',i,'\t  threshold: ','%.3f'% tr ,'  precision: ', '%.3f' % precision,'  recall: ', '%.3f' % recall) 


# NEXT: find the best threshold and try the model on the test set!


In [ ]:
#This part computes the ROC curves for both models like we talked about in class

from sklearn.metrics import roc_curve
from sklearn.mixture import GaussianMixture

gm_sf = GaussianMixture(n_components = 1, covariance_type = 'full', random_state=0, )
gm_sf.fit(X_train)
p = gm_sf.score_samples(X_train)

fpr_sc, tpr_sc, _ = roc_curve(y_train, -1 * p)
fpr_gm, tpr_gm, _ = roc_curve(y_train, -1 * p_gm)
plt.plot(fpr_sc, tpr_sc, linestyle = '--', label='Single Component')
plt.plot(fpr_gm, tpr_gm, marker='.', label='Gaussian Mixture')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import auc
print ("AUC of Single Component" , format( auc(fpr_sc, tpr_sc) , ".3f"))
print ("AUC of Gaussian Mixture" , format( auc(fpr_gm, tpr_gm) , ".3f"))

In [ ]:
from sklearn.metrics import roc_auc_score
print ("AUC of Single Component" , format(  roc_auc_score(y_train, -1 * p)     , ".3f")  )
print ("AUC of Gaussian Mixture" , format(  roc_auc_score(y_train, -1 * p_gm)  , ".3f")  )

In [ ]:
#This part computes the ROC curves for both models like we talked about in class

from sklearn.metrics import roc_curve
from sklearn.mixture import GaussianMixture

gm2 = GaussianMixture(n_components = 10, covariance_type = 'full', random_state=0, )
gm2.fit(X_train)
p_gm2 = gm2.score_samples(X_train)

fpr_sc, tpr_sc, _ = roc_curve(y_train, -1 * p)
fpr_gm, tpr_gm, _ = roc_curve(y_train, -1 * p_gm)
fpr_gm2, tpr_gm2, _ = roc_curve(y_train, -1 * p_gm2)
plt.plot(fpr_sc, tpr_sc, linestyle = '--', label='Single Component')
plt.plot(fpr_gm, tpr_gm, marker='.', label='Gaussian Mix 5 cmp')
plt.plot(fpr_gm2, tpr_gm2, marker='*', label='Gaussian Mix 10 cmp')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

Let's look at a dataset that motivates using a Mixture of Gaussians model: Simpsons ratings.

Everyone knows that there's a certain point when The Simpsons "got bad", but can we use a Mixture of Gaussians to find out exactly when that was?

Load up the `simpsons.pickle` file using the cell below. It contains the IMDb rating for every simpsons episode.

In [ ]:
!wget https://github.com/alexwolson/APS1070_data/raw/master/simpsons.pickle
import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GaussianMixture
with open('simpsons.pickle','rb') as f:
    simpsons = pickle.load(f)

Plot a histogram of the rating distribution for all Simpsons episodes. 


In [ ]:
allratings = []
for v in simpsons.values():
    for v1 in v.values():
        allratings.append(v1)
plt.hist(allratings, bins= 30)
plt.show()

Next, use Gaussian Mixture to fit a Mixture of Gaussians to the Simpsons rating distribution. Since we are trying to distinguish between good and bad ratings, we only need 2 gaussians.

* What are the means for the two Gaussians fit by the model? __
* What about the standard deviations? __

In [ ]:
gm = GaussianMixture(n_components=2)
gm.fit(np.array(allratings).reshape(-1,1))

print("Mean: ", gm.means_)
print(" \n \n StD: ", np.sqrt(gm.covariances_))


In [ ]:
from matplotlib import rc
from sklearn import mixture
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import matplotlib.ticker as tkr
import scipy.stats as stats


x = np.array(allratings)

f = np.ravel(x).astype(np.float)
f=f.reshape(-1,1)

weights = gm.weights_
means = gm.means_
covars = gm.covariances_

plt.hist(f, bins=30, histtype='bar', density=True, ec='blue', alpha=0.5)

f_axis = f.copy().ravel()
f_axis.sort()
plt.plot(f_axis,weights[0]*stats.norm.pdf(f_axis,means[0],np.sqrt(covars[0])).ravel(), c='red', label = "G1")
plt.plot(f_axis,weights[1]*stats.norm.pdf(f_axis,means[1],np.sqrt(covars[1])).ravel(), c='blue', label = "G2")
plt.legend()
plt.xlabel("Rate")
plt.ylabel("Dist")
plt.show()

Finally, using the `GuassianMixture.predict()` method, we can use maximum likelihood to estimate which distribution, good or bad, each episode belongs to. In the cell below, we have provided code to count the number of episodes predicted to be in the "good" distribution per season, and plot for the same.
Understand the code and answer the question.

* Where is the notable drop-off point? __
* What is the first season with 0 good episodes? __


In [ ]:
#Let's first associate each component with a good or bad season
if gm.means_[0,0] > gm.means_[1,0]: #True if first component is the good season (ie, higher mean)
     Good_season_index = 0
else:
     Good_season_index = 1

Xs = []
Ys = []
simpsons = dict(sorted(list(simpsons.items()), key=lambda x: x[0]))
for season, episodes in simpsons.items():
     bad = 0
     good = 0
     for episode in episodes.values():
          if gm.predict(np.array(episode).reshape(-1,1)) == Good_season_index:
               good += 1
          else:
               bad += 1
     Xs.append(season)
     Ys.append(good/(good+bad))
plt.plot(Xs, Ys, lw = 4, color = "green");
plt.xlabel("Seasons")
plt.ylabel("Good ratio")
plt.show()